# Identificamos skills tecnológicas a través de las descripciones laborales
## Intro
#### Mediante un asistente creado en la [OpenAI developer platform](https://platform.openai.com/) vamos a extraer de las descripciones laborales, la información que queremos con el formato que he indicado al asistente, en este caso un JSON.

#### Esto (mediante entrenamiento) nos ahorra problemas como:
- Tecnologías escritas de diferente forma. (PowerBi, Power BI o Powerbi por ejemplo)
- El idioma de las descripciones.
- Filtros enormes que habría que hacer para el tratamiento de texto (tildes, mayusculas, etc).


### Formato del Json resultante
<img src="../recursos/json_extraido_por_model_ETL.png" height="500px">

---
---

## Importamos librerías y configuraciones

In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import ast
from tqdm import tqdm
import pandas as pd
tqdm.pandas()

%load_ext autoreload
%autoreload 2

load_dotenv()

True

## creamos diccionario de dataframes con las ofertas

In [2]:
dic_dfs_ofertas = {archivo.rstrip('.csv') : pd.read_csv(f'../datos/datos_recibidos_empresa/{archivo}') for archivo in os.listdir('../datos/datos_recibidos_empresa/') if archivo.endswith('.csv')}

dic_dfs_ofertas.keys()

dict_keys(['ofertas_data_analyst', 'ofertas_data_engineer', 'ofertas_data_scientist', 'ofertas_seguridad_privacy_data'])

## Conectamos con api OpenAI y realizamos la extracion de skills

In [3]:
api_openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# asistente creado y entrenado en el playground de la api
asistente_extractor = api_openai.beta.assistants.retrieve(os.getenv('ASSISTANT_API_KEY'))


def extraer_skills_descripcion(descripcion):
    """
    Función para extraer habilidades a partir de una descripción utilizando un asistente de OpenAI.
    Por cada ejecución se crea una nueva conversacion para no tener que mantener el contexto de la conversación,
    siendo más eficiente a nivel de tokens y pero menos eficiente a nivel de codigo.
    """
    try:
        # Crear una nueva conversación
        try:
            conversacion = api_openai.beta.threads.create()
        except Exception as e:
            print(f"Error al crear la conversación: {e}")
            return ['Error al iniciar la conversación']

        # Enviar el mensaje del usuario al hilo de conversación
        try:
            mensaje = api_openai.beta.threads.messages.create(
                thread_id=conversacion.id,
                role="user",
                content=descripcion
            )
        except Exception as e:
            print(f"Error al enviar el mensaje: {e}")
            return ['Error al enviar el mensaje']

        # Ejecutar y esperar la respuesta del asistente
        try:
            ejecucion_asistente = api_openai.beta.threads.runs.create_and_poll(
                thread_id=conversacion.id,
                assistant_id=asistente_extractor.id
            )
        except Exception as e:
            print(f"Error durante la ejecución del asistente: {e}")
            return ['Error durante la ejecución del asistente']

        # Verificar el estado de la ejecución
        if ejecucion_asistente.status == 'completed':
            try:
                # Obtener todos los mensajes del hilo
                mensajes = api_openai.beta.threads.messages.list(
                    thread_id=conversacion.id
                )
                # Procesar la respuesta del asistente
                respuesta = ast.literal_eval(mensajes.data[0].content[0].text.value)
                return respuesta['skills'] if respuesta else ['Desconocidas']
            except Exception as e:
                print(f"Error al procesar la respuesta: {e}")
                return ['Error al procesar la respuesta']
        else:
            # Si el estado no fue completado, devolver el estado como mensaje
            print(f"Estado de la ejecución: {ejecucion_asistente.status}")
            return ['Estado no completado']

    except Exception as e:
        # Capturar cualquier error inesperado
        print(f"Error inesperado: {e}")
        return ['Error inesperado']

In [4]:
for key, value in tqdm(dic_dfs_ofertas.items()):
    print(f'Obteniendo info: {key.upper()}\n')

    df = value
    df['Skills'] = df['Descripcion'].progress_map(lambda x: extraer_skills_descripcion(x))

    df.to_csv(f'../datos/datos_recibidos_empresa/datos_skills_identificadas/{key}_identificadas.csv', index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

Obteniendo info: OFERTAS_DATA_ANALYST



Estado de la ejecución: failed


Estado de la ejecución: failed


 25%|██▌       | 1/4 [33:44<1:41:14, 2024.70s/it]

...

Obteniendo info: OFERTAS_DATA_SCIENTIST



 75%|███████▌  | 3/4 [1:41:47<34:03, 2043.25s/it]  

Obteniendo info: OFERTAS_SEGURIDAD_PRIVACY_DATA



Estado de la ejecución: failed


100%|██████████| 4/4 [2:17:13<00:00, 2058.27s/it]
